This is the first draft where the goal will be to have the preprocessor feed the trained neural network a face, and for an output to be given. The user should be able to take a picture and get a reading.

In [1]:
###Import Block
import pandas as pd
import numpy as np
import math
import cv2
import tensorflow as tf
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics as mt
from sklearn.metrics import roc_curve
from skimage.io import imshow
from matplotlib import pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.metrics import auc
import seaborn as sns
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 

#print entire array
import sys
np.set_printoptions(threshold=sys.maxsize)

### Read Faces Datasets

In [ ]:
df = pd.read_csv('faces.csv')
#df = df.drop(columns=['Usage'])
print(df.shape)
df.head()

In [ ]:
### Limit the categories to just happy, sad, neutral, and prepare data
#label_to_hr = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}
#0: anger
#1: disgust (almost no entries)
#2: fear (not as many, getting confused with sadness)
#3: happiness
#4: sadness
#5: surprise
#6: neutral
LabelsOfInterest = [3, 4, 5] # happy, sad, surprised

NUM_CLASSES = len(LabelsOfInterest)
df = df[df.emotion.isin(LabelsOfInterest)]
print(df.head())
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array = np.stack(img_array, axis=0)
le = LabelEncoder()
img_labels = le.fit_transform(df.emotion)
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

X_train, X_test, y_train, y_test = train_test_split(img_array, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.2, random_state=42)
img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]
X_train = (X_train/255)-0.5
X_test = (X_test/255)-0.5
X_test.shape
y_train.shape

### Building Model

In [ ]:
##Chosen Architecture: Use two convolution layers
l2_lambda = 0.0001
cnn = Sequential() 

cnn.add( Conv2D(filters=32, 
                kernel_size= (3, 3),
                kernel_initializer='he_uniform', 
                kernel_regularizer=l2(l2_lambda),
                padding='same', 
                input_shape=(img_width, img_height, img_depth),
                activation='relu',
                data_format="channels_last") )

# cnn.add( MaxPooling2D(pool_size=(2, 2), 
#                 data_format="channels_last") )

cnn.add( Conv2D(filters=32, 
                kernel_size= (3, 3),
                kernel_initializer='he_uniform', 
                kernel_regularizer=l2(l2_lambda),
                padding='same', 
                input_shape=(img_width, img_height, img_depth),
                activation='relu',
                data_format="channels_last") )

cnn.add( MaxPooling2D(pool_size=(2, 2), 
                data_format="channels_last") )

cnn.add( Conv2D(filters=64, 
                kernel_size= (3, 3),
                kernel_initializer='he_uniform', 
                kernel_regularizer=l2(l2_lambda),
                padding='same', 
                input_shape=(img_width, img_height, img_depth),
                activation='relu',
                data_format="channels_last") )

# cnn.add( MaxPooling2D(pool_size=(2, 2), 
#                 data_format="channels_last") )

cnn.add( Conv2D(filters=64, 
                kernel_size= (3, 3),
                kernel_initializer='he_uniform', 
                kernel_regularizer=l2(l2_lambda),
                padding='same', 
                input_shape=(img_width, img_height, img_depth),
                activation='relu',
                data_format="channels_last") )

cnn.add( MaxPooling2D(pool_size=(2, 2), 
                data_format="channels_last") )

cnn.add(Conv2D(filters=128,
               input_shape=(img_width, img_height, img_depth),
               kernel_size=(3,3),
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda),
               padding='same', 
               activation='relu',data_format="channels_last")) # more compact syntax

cnn.add(Conv2D(filters=128,
               kernel_size=(3,3),
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda),
               padding='same', 
               activation='relu',data_format="channels_last"))


# add one layer on flattened output
cnn.add( Flatten() )
cnn.add(Dropout(0.25))
cnn.add(Dense(128, 
              activation='relu',
              kernel_initializer='he_uniform',
              kernel_regularizer=l2(l2_lambda) ))

cnn.add( Dense(NUM_CLASSES) )
cnn.add( Activation('softmax') )

cnn.summary()

In [ ]:
%%time 

# Let's train the model 
cnn.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = cnn.fit(X_train, y_train, 
        batch_size=32, epochs=50,
        shuffle=True, verbose=1,
        validation_data=(X_test,y_test),
        callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

In [ ]:
# Save trained model
cnn.save('cnn_model')

### Test Code

In [6]:
#Preprocessor
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
img = cv2.imread("TestEmotions/input_happy.jpg")
displayImg = cv2.resize(img, (495, 660))
cv2.imshow("Show", displayImg)
#cv2.waitKey(0)
print('2')
faces = faceCascade.detectMultiScale(img,1.1,4)
facePics =[]
facePics.clear()
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h), (0,255,0),2)
    tempface = img[y:y+h, x:x+w]
    facePics.append(tempface)
imgResize = cv2.resize(facePics[0],(48,48))
imgGray = cv2.cvtColor(imgResize, cv2.COLOR_BGR2GRAY)
cv2.imshow("Show", imgGray)
cv2.waitKey(0)

2


-1

In [ ]:
print("Original Image Shape: " + str(imgGray.shape))
print("NetInput Image Shape: " + str(X_test.shape))
reshapeImg = imgGray.reshape((1,48,48,1))
print("Working Image Shape:  " + str(reshapeImg.shape))

In [ ]:
normalizedImg = (reshapeImg - np.amin(reshapeImg)) / (np.amax(reshapeImg) - np.amin(reshapeImg))
normalizedImg = 2*normalizedImg-1 #s
print("0 is Anger, 1 is Afraid, 2 is Happy, 3 is Sad")
yhat_cnn = np.argmax(cnn.predict(normalizedImg), axis=1)
print(yhat_cnn)

In [ ]:
def plot_results(cnn, X_test, y_test):
    yhat_cnn = np.argmax(cnn.predict(X_test), axis=1)
    print(yhat_cnn)
    acc_cnn = mt.accuracy_score(np.argmax(y_test, axis=1),yhat_cnn)
    cm = mt.confusion_matrix(np.argmax(y_test, axis=1),yhat_cnn)
    cm = cm/np.sum(cm,axis=1)[:,np.newaxis]
    sns.heatmap(cm, annot=True, fmt='.2f')
    plt.title('Results: '+str(acc_cnn))
    plt.show()

In [ ]:
plot_results(cnn, X_test, y_test)